In [1]:
import pandas as pd
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS as stopwords
import re
import string
from nltk.tokenize import word_tokenize
from textblob import TextBlob,Word
from nltk.corpus import words,wordnet
from nltk.stem import WordNetLemmatizer

In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

**Open Dataset**

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TUGAS AKHIR /tweets.csv', encoding='latin1')
df

,tweets,sentiment,label
0,Many Americans lack Critical Thinking Skills! ...,negative,-1
1,Coronavirus: Germany has a problem with the va...,negative,-1
2,You cannot get vaxxed when you already have CO...,negative,-1
3,Shame on the provaxxers for their judgemental ...,negative,-1
4,Someone died because of vaccine,negative,-1
...,...,...,...
3745,"Vaccination with Sputnik V starts in Patna, In...",positive,1
3746,"thanks to a great team, truly a blessing.",positive,1
3747,Enthusiastic Vaccine Booster,positive,1
3748,the Pfizer vaccine is now authorized for 12-15...,positive,1


In [4]:
df['sentiment'].value_counts()

positive    1250
negative    1250
neutral     1250
Name: sentiment, dtype: int64

# Lowercase Conversion

In [5]:
df['tweets'] = df['tweets'].apply(lambda x : str(x).lower())

In [6]:
df.sample(5)

,tweets,sentiment,label
2943,i was lucky,positive,1
3486,feeling very optimistic,positive,1
3416,i just booked my vaccine booster for next week...,positive,1
1511,thinking of getting the covid vaccine? here's ...,neutral,0
1843,covid-19 vaccination centre in. check out the ...,neutral,0


# Contraction to Expasion

In [7]:
contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "can not",
"can't've": "can not have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how're": "how are",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "I would",
"i'd've": "I would have",
"i'll": "I will",
"i'll've": "I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you shall have",
"you're": "you are",
"you've": "you have"
}

In [8]:
def cont_to_exp(x):
    if type(x) is str:
        for key in contractions:
            value = contractions[key]
            x = x.replace(key, value)
        return x 
    else:
        return x

In [9]:
ex = "ain't happy"
cont_to_exp(ex)

'am not happy'

In [10]:
%%timeit
df['tweets'] = df['tweets'].apply(lambda x : cont_to_exp(x))

10 loops, best of 5: 131 ms per loop


In [11]:
df.sample(5)

,tweets,sentiment,label
1950,read more on the new year's eve 80% vaccinatio...,neutral,0
1225,covid vaccine i hate you so bad,negative,-1
1951,the united arab emirates authorizes the sputni...,neutral,0
2767,"interesting... very, very interesting...??????...",positive,1
3052,some good news for covid vaccine,positive,1


# Remove RT

In [12]:
df[df['tweets'].str.contains('rt @')]

,tweets,sentiment,label
1528,b'rt @surreyplaceon: people with intellectual ...,neutral,0
2963,rt @rainnwilson: i love science and am so grat...,positive,1


In [13]:
x = 'rt@username: hello hirt'

In [14]:
re.sub(r'\brt\b', '', x).strip()

'@username: hello hirt'

In [15]:
y = "b'rt @surreyplaceon: people hirt, rt @username: hello hirt"
re.sub(r"\bb'rt\b|\brt\b", "", y).strip()

'@surreyplaceon: people hirt,  @username: hello hirt'

In [16]:
df['tweets'] = df['tweets'].apply(lambda x: re.sub(r"\bb'rt\b |\brt\b", "", x).strip())

In [17]:
df.iloc[[1528]]

,tweets,sentiment,label
1528,@surreyplaceon: people with intellectual &amp;...,neutral,0


In [18]:
df.iloc[[2963]]

,tweets,sentiment,label
2963,@rainnwilson: i love science and am so gratefu...,positive,1


# Remove multiple spaces

In [19]:
x =  'hi    hello     how are you'

In [20]:
' '.join(x.split())

'hi hello how are you'

In [21]:
df['tweets'] = df['tweets'].apply(lambda x: ' '.join(x.split()))

In [22]:
df

,tweets,sentiment,label
0,many americans lack critical thinking skills! ...,negative,-1
1,coronavirus: germany has a problem with the va...,negative,-1
2,you cannot get vaxxed when you already have co...,negative,-1
3,shame on the provaxxers for their judgemental ...,negative,-1
4,someone died because of vaccine,negative,-1
...,...,...,...
3745,"vaccination with sputnik v starts in patna, in...",positive,1
3746,"thanks to a great team, truly a blessing.",positive,1
3747,enthusiastic vaccine booster,positive,1
3748,the pfizer vaccine is now authorized for 12-15...,positive,1


# Remove Accented Chars

In [23]:
x = 'Áccěntěd těxt'

In [24]:
import unicodedata

In [25]:
def remove_accented_chars(x):
    x = unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return x

In [26]:
remove_accented_chars(x)

'Accented text'

In [27]:
df['tweets'] = df['tweets'].apply(lambda x: remove_accented_chars(x))

In [28]:
df

,tweets,sentiment,label
0,many americans lack critical thinking skills! ...,negative,-1
1,coronavirus: germany has a problem with the va...,negative,-1
2,you cannot get vaxxed when you already have co...,negative,-1
3,shame on the provaxxers for their judgemental ...,negative,-1
4,someone died because of vaccine,negative,-1
...,...,...,...
3745,"vaccination with sputnik v starts in patna, in...",positive,1
3746,"thanks to a great team, truly a blessing.",positive,1
3747,enthusiastic vaccine booster,positive,1
3748,the pfizer vaccine is now authorized for 12-15...,positive,1


#   Clean Text

In [33]:
special_char=re.compile(r'[-%<>.?!,;#:/\\()^~`$₹+\'*&_{}=|0-9]')

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def clean_text(tweet): 
    global special_char
    lemmatizer = WordNetLemmatizer() 
    tweet = re.sub('@.*? |#.*? |@.*?: |@.*? |@.*?:|#.*?: |@.*?:|#.*?:', '', tweet)
    #tweet = re.sub("[^a-zA-Z]", " ", tweet)
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
    tweet = re.sub(r'https?:\/\/.*|http?:\/\/.*',"", tweet)
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'"', '', tweet)
    tweett = re.sub(r'amp', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    tweet = re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , tweet)
    tweet = special_char.sub(r" ",tweet)

    word_tokens = word_tokenize(tweet)

    filtered_tweet = []
    for w in word_tokens:
        if ((w not in string.punctuation)  and (len(w)>1)):
            filtered_tweet.append(w)

    
    filtered_tweets=[lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in filtered_tweet]        
    filtered_sentence = ' '.join(filtered_tweets)
    return filtered_sentence

In [34]:
df['tweets'] = df['tweets'].apply(clean_text)
df

,tweets,sentiment,label
0,many american lack critical think skill stop l...,negative,-1
1,coronavirus germany have problem with the vacc...,negative,-1
2,you can not get vaxxed when you already have c...,negative,-1
3,shame on the provaxxers for their judgemental ...,negative,-1
4,someone die because of vaccine,negative,-1
...,...,...,...
3745,vaccination with sputnik start in patna india ...,positive,1
3746,thanks to great team truly blessing,positive,1
3747,enthusiastic vaccine booster,positive,1
3748,the pfizer vaccine be now authorize for year o...,positive,1


# Upload

In [35]:
df = df[['tweets', 'sentiment', 'label']]
df

,tweets,sentiment,label
0,many american lack critical think skill stop l...,negative,-1
1,coronavirus germany have problem with the vacc...,negative,-1
2,you can not get vaxxed when you already have c...,negative,-1
3,shame on the provaxxers for their judgemental ...,negative,-1
4,someone die because of vaccine,negative,-1
...,...,...,...
3745,vaccination with sputnik start in patna india ...,positive,1
3746,thanks to great team truly blessing,positive,1
3747,enthusiastic vaccine booster,positive,1
3748,the pfizer vaccine be now authorize for year o...,positive,1


In [36]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/TUGAS AKHIR /fixed_tweets.csv', encoding='latin1', index=False)
df

,tweets,sentiment,label
0,many american lack critical think skill stop l...,negative,-1
1,coronavirus germany have problem with the vacc...,negative,-1
2,you can not get vaxxed when you already have c...,negative,-1
3,shame on the provaxxers for their judgemental ...,negative,-1
4,someone die because of vaccine,negative,-1
...,...,...,...
3745,vaccination with sputnik start in patna india ...,positive,1
3746,thanks to great team truly blessing,positive,1
3747,enthusiastic vaccine booster,positive,1
3748,the pfizer vaccine be now authorize for year o...,positive,1
